In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from braintumorClassifier.constants import *
from braintumorClassifier.utils.common import read_yaml, create_directories


[2024-02-08 19:37:18,102: INFO: __init__: Starting data processing]
[2024-02-08 19:37:18,103: INFO: __init__: Data preprocessing completed successfully.]


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion
            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                 root_dir=config.root_dir,
                 source_url=config.source_url,
                 local_data_file=config.local_data_file,
                 unzip_dir=config.unzip_dir
            )

            return data_ingestion_config
        



In [8]:
import os
import urllib.request as request
import zipfile
from braintumorClassifier import logger
from braintumorClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f'Downloaded {filename} with following headers: \n{header}')
        else:
            logger.info(f'File already exists of size : {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        Unzips the downloaded file
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f'File extracted to: {unzip_path}')


In [23]:
%pwd

'c:\\Users\\Dell\\Documents'

In [10]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-08 19:37:30,443: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-08 19:37:30,445: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-08 19:37:30,448: INFO: common: created directory at: artifacts]
[2024-02-08 19:37:30,450: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-08 19:39:57,475: INFO: 1905394879: Downloaded artifacts/data_ingestion/data.zip with following headers: 
Connection: close
Content-Length: 19883640
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e9099c6112beae9f2a0d5fe26cfaa4c4caa0fc27be045c6f81bf2992d42b9bf4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3572:1274B0:222C83:2B4B9D:65C4DDF7
Accept-Ranges: bytes
Date: Thu, 08 Feb 2024 14:37:31 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1228-QPG
X-Cach